### Data Preparation:
This part of the project is inspired by the great work of Andrew Long.

https://towardsdatascience.com/introduction-to-clinical-natural-language-processing-predicting-hospital-readmission-with-1736d52bc709

I am preparing data exactly the same way as him so that at the end I can compare my results with his.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import pickle

#### Reading Admission Data

In [17]:
df_adm = pd.read_csv('ADMISSIONS.csv', parse_dates = ['ADMITTIME', 'DISCHTIME',
       'DEATHTIME'], infer_datetime_format = True, engine = "c")

In [18]:
df_adm = df_adm[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE']]

In [19]:
df_adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 6 columns):
SUBJECT_ID        58976 non-null int64
HADM_ID           58976 non-null int64
ADMITTIME         58976 non-null datetime64[ns]
DISCHTIME         58976 non-null datetime64[ns]
DEATHTIME         5854 non-null datetime64[ns]
ADMISSION_TYPE    58976 non-null object
dtypes: datetime64[ns](3), int64(2), object(1)
memory usage: 2.7+ MB


#### The next step is to get the next unplanned admission date if it exists. Sort by SUBJECT_ID and ADMITTIME.

In [20]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)

In [21]:
df_adm.tail()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE
58971,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,NaT,EMERGENCY
58972,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,ELECTIVE
58973,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,NaT,EMERGENCY
58974,99995,137810,2147-02-08 08:00:00,2147-02-11 13:15:00,NaT,ELECTIVE
58975,99999,113369,2117-12-30 07:15:00,2118-01-04 16:30:00,NaT,ELECTIVE


#### It is the way I want it to be

In [22]:
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)

#### Get the next admission type

In [23]:
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

#### Lets see how it looks!

In [24]:
df_adm.head(200)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NaT,NaN
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NaT,NaN
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN
...,...,...,...,...,...,...,...,...
195,146,190707,2119-12-19 12:15:00,2120-01-10 13:08:00,NaT,URGENT,NaT,NaN
196,147,103631,2158-06-24 18:50:00,2158-07-21 14:10:00,NaT,EMERGENCY,NaT,NaN
197,148,199488,2107-09-05 14:58:00,2107-10-08 13:35:00,NaT,EMERGENCY,NaT,NaN
198,149,154869,2135-02-18 19:16:00,2135-02-26 17:00:00,NaT,EMERGENCY,NaT,NaN


#### So for patients who have more than one admission we have the next date and type of admission. We want to predict "UNPLANNED" re-admissions, so we should filter out next admissions which are "ELECTIVE". Therefore, we get rows where next admission is elective and replace next date and next type with naT and nan respectively.

In [25]:
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [26]:
df_adm.head(200)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NaT,NaN
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NaT,NaN
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN
...,...,...,...,...,...,...,...,...
195,146,190707,2119-12-19 12:15:00,2120-01-10 13:08:00,NaT,URGENT,NaT,NaN
196,147,103631,2158-06-24 18:50:00,2158-07-21 14:10:00,NaT,EMERGENCY,NaT,NaN
197,148,199488,2107-09-05 14:58:00,2107-10-08 13:35:00,NaT,EMERGENCY,NaT,NaN
198,149,154869,2135-02-18 19:16:00,2135-02-26 17:00:00,NaT,EMERGENCY,NaT,NaN


#### For example, for rows 180 and 185 the elective readmissions are deleted. Again I sort by subject_ID and admission date before the fill.

In [27]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
# back fill (this will take a little while)
df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

In [28]:
df_adm.head(200)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NaT,NaN
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NaT,NaN
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN
...,...,...,...,...,...,...,...,...
195,146,190707,2119-12-19 12:15:00,2120-01-10 13:08:00,NaT,URGENT,NaT,NaN
196,147,103631,2158-06-24 18:50:00,2158-07-21 14:10:00,NaT,EMERGENCY,NaT,NaN
197,148,199488,2107-09-05 14:58:00,2107-10-08 13:35:00,NaT,EMERGENCY,NaT,NaN
198,149,154869,2135-02-18 19:16:00,2135-02-26 17:00:00,NaT,EMERGENCY,NaT,NaN


#### Time difference between two admissions in days:

In [29]:
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

In [30]:
df_adm.head(200)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NaT,NaN,NaN
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN,NaN
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN,NaN
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NaT,NaN,NaN
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...
195,146,190707,2119-12-19 12:15:00,2120-01-10 13:08:00,NaT,URGENT,NaT,NaN,NaN
196,147,103631,2158-06-24 18:50:00,2158-07-21 14:10:00,NaT,EMERGENCY,NaT,NaN,NaN
197,148,199488,2107-09-05 14:58:00,2107-10-08 13:35:00,NaT,EMERGENCY,NaT,NaN,NaN
198,149,154869,2135-02-18 19:16:00,2135-02-26 17:00:00,NaT,EMERGENCY,NaT,NaN,NaN


#### Reading NOTEEVENTS table:

In [31]:
use_cols = ['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'TEXT']

In [32]:
df_notes = pd.read_csv("NOTEEVENTS.csv",usecols= use_cols, low_memory = False, engine = "c")

#### I only want discharge summarys:

In [34]:
categories = df_notes.CATEGORY.value_counts()
print(categories)

Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: CATEGORY, dtype: int64


In [48]:
df_notes_dis_sum = df_notes.loc[df_notes.CATEGORY == 'Discharge summary']
print(len(df_notes_dis_sum))

59652


#### Some patients have more than one discharge summary in one admission. We can verify that:

In [39]:
assert df_notes_dis_sum.duplicated(['HADM_ID']).sum() == 0

AssertionError: 

#### For simplicity I am going to use only the last discharge note ( and also because Andrew Long has used only last ones):

In [40]:
df_notes_dis_sum_last = (df_notes_dis_sum.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()

#### Make sure only one discharge note per admission exists:

In [41]:
assert df_notes_dis_sum_last.duplicated(['HADM_ID']).sum() == 0, 'Multiple discharge summaries per admission'

#### Joining two tables:

In [42]:
df_adm_notes = pd.merge(df_adm, df_notes_dis_sum_last, on = ['SUBJECT_ID','HADM_ID'],how = 'left')

In [43]:
assert len(df_adm) == len(df_adm_notes), 'Number of rows increased'

In [44]:
df_adm_notes.TEXT.isnull().sum() / len(df_adm_notes)

0.1059753119913185

#### 10.6 % of the admissions are missing 

In [45]:
df_adm_notes.groupby('ADMISSION_TYPE').apply(lambda g: g.TEXT.isnull().sum())/df_adm_notes.groupby('ADMISSION_TYPE').size()

ADMISSION_TYPE
ELECTIVE     0.048663
EMERGENCY    0.037983
NEWBORN      0.536691
URGENT       0.042665
dtype: float64

#### 53% of the NEWBORN admissions were missing discharge summaries vs ~4% for the others. Therfore for now I am not using NEWBORN category.

In [46]:
df_adm_notes_clean = df_adm_notes.loc[df_adm_notes.ADMISSION_TYPE != "NEWBORN"] 

#### Making the response variable:

In [47]:
df_adm_notes_clean['OUTPUT_LABEL'] = (df_adm_notes_clean.DAYS_NEXT_ADMIT < 30).astype('int')

In [31]:
df_adm_notes_clean.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT,CATEGORY,TEXT,OUTPUT_LABEL
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2101-10-20**] Discharg...,0
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2191-3-16**] Discharge...,0
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN,NaN,Discharge summary,Admission Date: [**2175-5-30**] Dischar...,0
7,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,NaT,NaN,NaN,Discharge summary,"Name: [**Known lastname 10050**], [**Known fi...",0
9,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2178-4-16**] ...,0


#### Let's check the number of patients in each class:

In [32]:
df_adm_notes_clean.OUTPUT_LABEL.value_counts()

0    48109
1     3004
Name: OUTPUT_LABEL, dtype: int64

In [33]:
df_adm_notes_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51113 entries, 1 to 58975
Data columns (total 12 columns):
SUBJECT_ID             51113 non-null int64
HADM_ID                51113 non-null int64
ADMITTIME              51113 non-null datetime64[ns]
DISCHTIME              51113 non-null datetime64[ns]
DEATHTIME              5792 non-null datetime64[ns]
ADMISSION_TYPE         51113 non-null object
NEXT_ADMITTIME         11169 non-null datetime64[ns]
NEXT_ADMISSION_TYPE    11169 non-null object
DAYS_NEXT_ADMIT        11169 non-null float64
CATEGORY               49083 non-null object
TEXT                   49083 non-null object
OUTPUT_LABEL           51113 non-null int64
dtypes: datetime64[ns](4), float64(1), int64(3), object(4)
memory usage: 5.1+ MB


#### Now I want to save the "df_adm_notes_clean" dataframe. To keep the format and type of the columns unchanged, I am using the pickle format.

In [34]:
df_adm_notes_clean.to_pickle('Prepared_Data.pkl')